In [1]:
from features import ensembleFeature
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import torch
from torch import nn
import re
from sklearn.utils import shuffle
from utils import metricsCal
from torch.utils.data import DataLoader,TensorDataset
import math
import sys
import copy
import pickle
from torch.autograd import Variable
from sklearn.model_selection import KFold
import torch.nn.functional as F
import os
import Attention_model as Model
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import csv
%matplotlib inline

In [2]:
#一个函数用于序列转换，一个函数用于检测“N”碱基
def long_short(data):
    seq_list = []
    for i in data[0]:
        seq_list.append(str(i)[480:521])
    return np.array(seq_list)

def check_N(data1,data2):
    seq_list = []
    gene_list = []
    for i in range(len(data1)):
        if str(data1[i]).find("N")<0:
            seq_list.append(data1[i])
            gene_list.append(data2[i])
    return np.array(seq_list),np.array(gene_list)

In [3]:
# type_name = ["FullTranscript","matureRNA"]
# cell_name = ["A549","CD8T","Hek293_abacm","Hek293_sysy","HeLa","MOLM13"]
type_name = ["matureRNA"]
cell_name = ["HeLa","MOLM13"]
#feature_name = ["emb","PSNP","PCP","DBPF"]
feature_name = ["PCP"]
network_name = ["BILSTM+Self-Attention","BILSTM+MultiSelf-Attention","BILSTM+Bah-Attention"]

rows = len(type_name)*len(cell_name)*len(feature_name)*len(network_name)*5
row = -1
max_epochs = 50
max_patience = 20
batch_size = 1024
row_name = []
Loss = np.zeros(rows*max_epochs).reshape(rows,max_epochs)
Acc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Mcc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Auc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
th_All = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))

In [4]:
Folder_name = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        Folder_list = []
        trainPosSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_train_"+type_name[typei]+"_seq.csv"
        trainNegSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_train_"+type_name[typei]+"_seq.csv"
        testPosSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_test_"+type_name[typei]+"_seq.csv"
        testNegSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_test_"+type_name[typei]+"_seq.csv"
        trainPosGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_train_"+type_name[typei]+"_gene.csv"
        trainNegGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_train_"+type_name[typei]+"_gene.csv"
        testPosGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_test_"+type_name[typei]+"_gene.csv"
        testNegGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_test_"+type_name[typei]+"_gene.csv"
        
        #获取文件信息
        trainPos_seq = pd.read_csv(trainPosSeq_file,header=None)
        trainNeg_seq = pd.read_csv(trainNegSeq_file,header=None)
        testPos_seq = pd.read_csv(testPosSeq_file,header=None)
        testNeg_seq = pd.read_csv(testNegSeq_file,header=None)
        trainPos_gene = pd.read_csv(trainPosGene_file) 
        trainNeg_gene = pd.read_csv(trainNegGene_file)
        testPos_gene = pd.read_csv(testPosGene_file)
        testNeg_gene = pd.read_csv(testNegGene_file)
        #序列转41-nt长度
        trainPos_seq = long_short(trainPos_seq)
        trainNeg_seq = long_short(trainNeg_seq)
        testPos_seq = long_short(testPos_seq)
        testNeg_seq = long_short(testNeg_seq)
        #基因转35长度
        trainPos_gene = trainPos_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        trainNeg_gene = trainNeg_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        testPos_gene = testPos_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        testNeg_gene = testNeg_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        trainPos_seq,trainPos_gene = check_N(trainPos_seq,np.array(trainPos_gene))
        trainNeg_seq,trainNeg_gene = check_N(trainNeg_seq,np.array(trainNeg_gene))
        testPos_seq,testPos_gene = check_N(testPos_seq,np.array(testPos_gene))
        testNeg_seq,testNeg_gene = check_N(testNeg_seq,np.array(testNeg_gene))
        #基因变Float
        trainPos_gene = np.float32(trainPos_gene)
        trainNeg_gene = np.float32(trainNeg_gene)
        testPos_gene = np.float32(testPos_gene)
        testNeg_gene = np.float32(testNeg_gene)
        #打乱顺序，再取等长
        trainNeg_seq, trainNeg_gene = shuffle(trainNeg_seq,trainNeg_gene, random_state=1)
        testNeg_seq, testNeg_gene = shuffle(testNeg_seq, testNeg_gene, random_state=1)

        trainNeg_seq = trainNeg_seq[:len(trainPos_seq)]
        trainNeg_gene = trainNeg_gene[:len(trainPos_gene)]
        testNeg_seq = testNeg_seq[:len(testPos_seq)]
        testNeg_gene = testNeg_gene[:len(testPos_gene)]
        
#         trainNeg_seq = trainNeg_seq[:len(trainPos_seq)*10]
#         trainNeg_gene = trainNeg_gene[:len(trainPos_gene)*10]
#         testNeg_seq = testNeg_seq[:len(testPos_seq)*10]
#         testNeg_gene = testNeg_gene[:len(testPos_gene)*10]
#         trainPos_seq = trainPos_seq.repeat(10)
#         trainPos_gene = trainPos_gene.repeat(10)
#         testPos_seq = testPos_seq.repeat(10)
#         testPos_gene = testPos_gene.repeat(10)

        #第二重循环
        for featurei in range(len(feature_name)):
            if feature_name[featurei] == "emb":
                trainPos = ensembleFeature.emb_seqs(trainPos_seq)
                trainNeg = ensembleFeature.emb_seqs(trainNeg_seq)
                testPos = ensembleFeature.emb_seqs(testPos_seq)
                testNeg = ensembleFeature.emb_seqs(testNeg_seq)
            elif feature_name[featurei] == "PSNP":
                trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSNP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq)
            elif feature_name[featurei] == "PCP":
                trainPos = ensembleFeature.PCP(trainPos_seq)
                trainNeg = ensembleFeature.PCP(trainNeg_seq)
                testPos = ensembleFeature.PCP(testPos_seq)
                testNeg = ensembleFeature.PCP(testNeg_seq)
            elif feature_name[featurei] == "DBPF":
                trainPos = ensembleFeature.DBPF(trainPos_seq)
                trainNeg = ensembleFeature.DBPF(trainNeg_seq)
                testPos = ensembleFeature.DBPF(testPos_seq)
                testNeg = ensembleFeature.DBPF(testNeg_seq)
            if len(testNeg.shape)==2:
                trainPos = trainPos.reshape(trainPos.shape[0],1,trainPos.shape[1])
                trainNeg = trainNeg.reshape(trainNeg.shape[0],1,trainNeg.shape[1])
                testPos = testPos.reshape(testPos.shape[0],1,testPos.shape[1])
                testNeg = testNeg.reshape(testNeg.shape[0],1,testNeg.shape[1])
#                 trainPos = trainPos.reshape(trainPos.shape[0],trainPos.shape[1],1)
#                 trainNeg = trainNeg.reshape(trainNeg.shape[0],trainNeg.shape[1],1)
#                 testPos = testPos.reshape(testPos.shape[0],testPos.shape[1],1)
#                 testNeg = testNeg.reshape(testNeg.shape[0],testNeg.shape[1],1)

            trainPos_label = np.ones(len(trainPos))
            trainNeg_label = np.zeros(len(trainNeg))
            testPos_label = np.ones(len(testPos))
            testNeg_label = np.zeros(len(testNeg))

            trainData = np.append(trainPos,trainNeg,axis=0)
            testData = np.append(testPos,testNeg,axis = 0)
            trainLabel = np.append(trainPos_label,trainNeg_label,axis = 0)
            testLabel = np.append(testPos_label,testNeg_label,axis = 0)

            kf = KFold(5,True,10)
            #第三重循环
            for networki in range(len(network_name)):
                if network_name[networki] == "BILSTM+Self-Attention":
                    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
#                         if i != 0:
#                             continue
                        row+=1
                        row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                        row_name.append(row_one)
                        X_train = trainData[train_index]
                        X_test = trainData[test_index]
                        Y_train = trainLabel[train_index]
                        Y_test = trainLabel[test_index]
                        best_auc = 0
                        patience = 0
                        model_dir = "Model_Attention_Extraction_Final/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                                feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                        model = Model.ModelBS_Pro(trainData.shape[1],trainData.shape[2])
                        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
                        
                        for j in range(max_epochs):
                            runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
                            acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
                            Loss[row][j] = runningLoss
                            th_All[row][j] = th
                            Acc[row][j] = acc
                            Mcc[row][j] = mcc
                            Auc[row][j] = auc
                           
                            if auc > best_auc:
                                best_auc=auc
                                patience=0
                            else:
                                patience+=1
                            if patience>max_patience and best_auc!= 0.5:
                                break
                        tmp_th = 0.5
                        tmp_auc = 0
                        for kk in range(max_epochs):
                            if Auc[row][kk]>tmp_auc:
                                tmp_th = th_All[row][kk]
                                tmp_auc = Auc[row][kk]
                        acc, mcc, auc=Model.independTest(testData,testLabel,device,model_dir,batch_size,tmp_th)
                        Acc[row][-1] = acc
                        Mcc[row][-1] = mcc
                        Auc[row][-1] = auc
                        th_All[row][-1] = tmp_th

                if network_name[networki] == "BILSTM+MultiSelf-Attention":
                    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
                        
                        row+=1
                        row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                        row_name.append(row_one)
                        X_train = trainData[train_index]
                        X_test = trainData[test_index]
                        Y_train = trainLabel[train_index]
                        Y_test = trainLabel[test_index]
                        best_auc = 0
                        patience = 0
                        model_dir = "Model_Attention_Extraction_Final/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                                feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                        model = Model.ModelB_MultiSelf_Pro(trainData.shape[1],trainData.shape[2])
                        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
                      
                        for j in range(max_epochs):
                            runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
                            acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
                            Loss[row][j] = runningLoss
                            th_All[row][j] = th
                            Acc[row][j] = acc
                            Mcc[row][j] = mcc
                            Auc[row][j] = auc
                            
                            if auc > best_auc:
                                best_auc=auc
                                patience=0
                            else:
                                patience+=1
                            if patience>max_patience and best_auc!= 0.5:
                                break
                        tmp_th = 0.5
                        tmp_auc = 0
                        for kk in range(max_epochs):
                            if Auc[row][kk]>tmp_auc:
                                tmp_th = th_All[row][kk]
                                tmp_auc = Auc[row][kk]
                        acc, mcc, auc=Model.independTest(testData,testLabel,device,model_dir,batch_size,tmp_th)
                        Acc[row][-1] = acc
                        Mcc[row][-1] = mcc
                        Auc[row][-1] = auc
                        th_All[row][-1] = tmp_th
                
                if network_name[networki] == "BILSTM+Bah-Attention":

                    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
#                         if i != 0:
#                             continue
                        row+=1
                        row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                        row_name.append(row_one)

                        X_train = trainData[train_index]
                        X_test = trainData[test_index]
                        Y_train = trainLabel[train_index]
                        Y_test = trainLabel[test_index]
                        best_auc = 0
                        patience = 0
                        model_dir = "Model_Attention_Extraction_Final/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                                feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                        model = Model.ModelB_Bah_Pro(trainData.shape[1],trainData.shape[2])
                        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
                        
                        for j in range(max_epochs):
                            runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
                            acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
                            Loss[row][j] = runningLoss
                            th_All[row][j] = th
                            Acc[row][j] = acc
                            Mcc[row][j] = mcc
                            Auc[row][j] = auc
                            
                            if auc > best_auc:
                                best_auc=auc
                                patience=0
                            else:
                                patience+=1
                            if patience>max_patience and best_auc!= 0.5:
                                 break
                        tmp_th = 0.5
                        tmp_auc = 0
                        for kk in range(max_epochs):
                            if Auc[row][kk]>tmp_auc:
                                tmp_th = th_All[row][kk]
                                tmp_auc = Auc[row][kk]
                        acc, mcc, auc=Model.independTest(testData,testLabel,device,model_dir,batch_size,tmp_th)
                        Acc[row][-1] = acc
                        Mcc[row][-1] = mcc
                        Auc[row][-1] = auc
                        th_All[row][-1] = tmp_th

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


10.399439632892609


/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.116
auc on test set: 0.575


/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


...

14.990453541278839


/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.289
auc on test set: 0.700


/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


0.5895405887600357 0.17985687078857826 0.6243590676454135
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.180
auc on test set: 0.624


In [11]:
# pd.DataFrame(np.append(np.array(row_name).reshape(len(row_name),1),Mcc,axis=1)).to_csv("Attention_Result/Mcc_PCP.csv",index=0)
# pd.DataFrame(np.append(np.array(row_name).reshape(len(row_name),1),Auc,axis=1)).to_csv("Attention_Result/Auc_PCP.csv",index=0)
# pd.DataFrame(np.append(np.array(row_name).reshape(len(row_name),1),Acc,axis=1)).to_csv("Attention_Result/Acc_PCP.csv",index=0)
# pd.DataFrame(np.append(np.array(row_name).reshape(len(row_name),1),th_All,axis=1)).to_csv("Attention_Result/th_PCP.csv",index=0)
# pd.DataFrame(np.append(np.array(row_name).reshape(len(row_name),1),Loss,axis=1)).to_csv("Attention_Result/Loss_PCP.csv",index=0)

In [7]:
pd.DataFrame(np.append(np.array(row_name).reshape(len(row_name),1),Mcc,axis=1)).to_csv("Attention_Result/Mcc_PCP_append.csv",index=0)
pd.DataFrame(np.append(np.array(row_name).reshape(len(row_name),1),Auc,axis=1)).to_csv("Attention_Result/Auc_PCP_append.csv",index=0)
pd.DataFrame(np.append(np.array(row_name).reshape(len(row_name),1),Acc,axis=1)).to_csv("Attention_Result/Acc_PCP_append.csv",index=0)
pd.DataFrame(np.append(np.array(row_name).reshape(len(row_name),1),th_All,axis=1)).to_csv("Attention_Result/th_PCP_append.csv",index=0)
pd.DataFrame(np.append(np.array(row_name).reshape(len(row_name),1),Loss,axis=1)).to_csv("Attention_Result/Loss_PCP_append.csv",index=0)

In [4]:
import pandas as pd
Auc = np.array(pd.read_csv("Attention_Result/Auc_PCP.csv").iloc[:,1:])
print(Auc.shape)

x1 = 0
x2 = 0
x3 = 0
for i in range(3):

    for j in range(12):
        if i == 0:
            x1+=np.mean(Auc[15*j+5*i:15*j+5*i+5,-1])
        if i == 1:
            x2+=np.mean(Auc[15*j+5*i:15*j+5*i+5,-1])
        if i == 2:
            x3+=np.mean(Auc[15*j+5*i:15*j+5*i+5,-1])
x1/12,x2/12,x3/12

(180, 51)


(0.6779548746213933, 0.6771204286397188, 0.6138322100508202)

In [ ]:
["BILSTM+Self-Attention","BILSTM+MultiHead-Attention","BILSTM+Bah-Attention"]